# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [33]:
# imports 
import os

from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import display, Markdown, update_display


In [8]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [14]:
# set up environment
load_dotenv(override=True)

# --- OpenRouter setup ---
openrouter_key = os.getenv("OPENROUTER_API_KEY")
openrouter_base = os.getenv("OPENROUTER_BASE_URL") or "https://openrouter.ai/api/v1"

if not openrouter_key:
    print("OpenRouter API key not found. Check your .env file.")
else:
    print("OpenRouter API key loaded.")

openai_client = OpenAI(
    api_key=openrouter_key,
    base_url=openrouter_base
)

# --- Ollama setup ---
OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama_client = OpenAI(
    base_url=OLLAMA_BASE_URL,
    api_key="ollama"
)

print("Environment setup complete.")

OpenRouter API key loaded.
Environment setup complete.


In [21]:
# here is the question; type over this to ask something new
system_role = """
You are a clear and precise technical assistant.
Explain code behavior accurately, including why it is written that way.
Use structured formatting where helpful.
"""

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [ ]:
# Get gpt-4o-mini to answer, with streaming 

print("GPT-4o-mini response:\n")

response = ""

stream = openai_client.chat.completions.create(
    model=MODEL_GPT,
    messages=[
        {"role": "system", "content": system_role},
        {"role": "user", "content": question}
    ],
    stream=True,
)

for chunk in stream:
    content = chunk.choices[0].delta.content
    if content:
        response += content

display(Markdown(response))

GPT-4o-mini response:



Certainly! Let's break down the code snippet you've provided:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

### Code Explanation

1. **Set Comprehension**:
   ```python
   {book.get("author") for book in books if book.get("author")}
   ```
   - This part of the code is a *set comprehension* that iterates over a collection called `books`.
   - For each `book` in `books`, it attempts to get the value associated with the `"author"` key using `book.get("author")`.
   - The `if book.get("author")` condition filters out any books where the `"author"` key either doesn't exist or has a value that evaluates as `False` (e.g., `None`, an empty string, etc.).
   - The result of this comprehension is a set containing the authors of the books that do have valid author entries.

2. **Yielding Values**:
   ```python
   yield from ...
   ```
   - The `yield from` statement is used in a generator function to yield all values from an iterable. In this case, the iterable is the set created by the comprehension.
   - This means that the generator will produce each author found in the set one at a time, allowing the calling context to iterate over these authors as they are generated.

### Why It Is Written This Way

- **Efficiency**: 
  - Using a set comprehension automatically removes any duplicate authors since sets do not allow duplicate entries. Therefore, this code will yield a unique list of authors.

- **Conditional Filtering**:
  - The `if book.get("author")` ensures that only books with a valid author are included, which is critical for preventing `None` or empty entries in the output.

- **Generator Functionality**:
  - By using `yield from`, you make the generator function capable of yielding each author without needing to individually loop through the set and yield each author manually. This makes the code more concise and maintains clear logic flow.

### Summary

In summary, this code defines a generator that yields unique authors from a list of `books`, filtering out any entries that do not contain valid author information. The set comprehension ensures uniqueness, while `yield from` facilitates efficient iteration over the resulting set.

In [34]:
# Get Llama 3.2 to answer
print("\n\nLlama response:\n")

response = ""
display_handle = display(Markdown(""), display_id=True)

stream = ollama_client.chat.completions.create(
    model=MODEL_LLAMA,
    messages=[
        {"role": "system", "content": system_role},
        {"role": "user", "content": question}
    ],
    stream=True,
)

for chunk in stream:
    content = chunk.choices[0].delta.content
    if content:
        response += content
        update_display(Markdown(response), display_id=display_handle.display_id)



Llama response:



**Code Explanation**

```python
yield from {book.get("author") for book in books if book.get("author")}
```

This is a Python code snippet that uses a combination of generators, dictionaries, and list comprehension. Here's what it does:

*   It extracts only the `author` field values from a list of books.
*   The extracted values are collected into a set (`{...}`).
*   If there are multiple authors for a book with no author (or an empty string), their contribution will be ignored.

**Breaking Down the Code**

1.  **Dictionary comprehension**: `{book.get("author") for book in books if book.get("author")}`:
    *   This creates a set containing `author` field values extracted from each book.
    *   Only books with authors are included (i.e., `if book.get("author")` filter) to avoid unnecessary processing.

2.  **Generator expression**: `yield from {...}`
    *   This takes the set created by the dictionary comprehension and yields its elements one by one.

3.  **Set used for deduplication and filtering**:
    *   The resulting generator's contribution is an empty iterable if there are duplicate `author` field values, which means we drop these duplicates.
    *   If a book has no `author`, it will not be included in the processed set.

**Example Usage**

Suppose you have a list of books where each book can be represented by a dictionary. Here's how you might use this code to iterate through authors:

```python
books = [
    {"title": "Book A", "author": "Author B"},
    {"title": "Book C", "author": "Author D"},
    {"title": "Book E", "",},  # No author provided
]

for author in yield from {book.get("author") for book in books if book.get("author")}:
    print(author)
```

In this example, the code will output `Author B` and `Author D`, but it won't include `Book E's` non-existent author.